In [1]:
import torch
from sympy.printing.tensorflow import tensorflow

In [2]:
pip install torch-geometric

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Dell\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [2]:
from torch_geometric.data import Data

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [55]:
data=torch.load('graph_data_with_txt.pt')

C:\Users\Dell\AppData\Local\Temp\ipykernel_31480\951303083.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data=torch.load('graph_data_with_txt.pt')


In [41]:
#data.y[data.y == -1] = 3

In [56]:
num_nodes = data.num_nodes
num_train = int(0.8 * num_nodes)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[:num_train] = 1

In [57]:
data.train_mask = train_mask
data.test_mask = ~train_mask

In [58]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn.functional import dropout
from torch_geometric.nn import GCNConv, Linear, SAGEConv

In [59]:
import torch
from torch_geometric.nn import GCNConv
from torch.nn import Linear, Dropout, LeakyReLU


In [95]:
import torch
from torch.nn import Linear, Dropout, Sigmoid
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(-1, 64)
        self.conv2 = GCNConv(64, 128)
        self.conv3 = GCNConv(128, 256)
        self.conv4 = GCNConv(256, 512)
        self.linear = Linear(512, 256)
        self.linear1=Linear(256,128)
        self.linear2 = Linear(128, 64)
        self.linear3 = Linear(64, 1)
        self.dropout = Dropout(p=0.4)

    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv2(x, edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv3(x,edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv4(x,edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        # x = self.conv5(x,edge_index)
        # x = torch.relu(x)
        # x = self.dropout(x)
        x=self.linear(x)
        x=self.linear1(x)
        x=self.linear2(x)
        x = self.linear3(x)
        return x

In [61]:
import torch
from torch.nn import Linear, Dropout, Sigmoid
from torch_geometric.nn import SAGEConv
class SAGE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SAGEConv(-1, 64)
        self.conv2 = SAGEConv(64, 128)
        self.conv3 = SAGEConv(128, 256)
        self.conv4 = SAGEConv(256, 512)
        self.linear = Linear(512, 256)
        self.linear1=Linear(256,128)
        self.linear2 = Linear(128, 64)
        self.linear3 = Linear(64, 1)
        self.dropout = Dropout(p=0.4)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv3(x,edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv4(x,edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        
        # x = self.conv5(x,edge_index)
        # x = torch.relu(x)
        # x = self.dropout(x)
        x=self.linear(x)
        x=self.linear1(x)
        x=self.linear2(x)
        x = self.linear3(x)
        return x

In [96]:
def train(model, dataset, optimizer, criterion, epochs=5):
    train_mask = dataset.train_mask
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(dataset.x, dataset.edge_index,dataset.edge_weight).squeeze()  # Output shape: [num_nodes]
        loss = criterion(out[train_mask], dataset.y[train_mask].float())  # Labels must be float
        loss.backward()
        optimizer.step()
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [122]:
data

Data(x=[4150, 1], edge_index=[2, 2270905], y=[4150], train_mask=[4150], test_mask=[4150])

In [109]:
model_gcn = GCN()

In [134]:
model_sage=SAGE()

In [64]:
model_gcn

GCN(
  (conv1): GCNConv(-1, 64)
  (conv2): GCNConv(64, 128)
  (conv3): GCNConv(128, 256)
  (conv4): GCNConv(256, 512)
  (linear): Linear(in_features=512, out_features=256, bias=True)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
)

In [98]:
optimizer = Adam(model_gcn.parameters(), lr=0.001)

In [99]:
criterion = torch.nn.BCEWithLogitsLoss()

In [15]:
(data.y == 0).sum().item()

3064

In [67]:
train(model=model_sage, dataset=data,
      criterion=criterion, optimizer=optimizer,
      epochs=50)

NameError: name 'model_sage' is not defined

In [100]:
train(model=model_gcn, dataset=data,
      criterion=criterion, optimizer=optimizer,
      epochs=50)

Epoch: 000, Loss: 0.6712
Epoch: 001, Loss: 0.6274
Epoch: 002, Loss: 0.6025
Epoch: 003, Loss: 0.6330
Epoch: 004, Loss: 0.6043
Epoch: 005, Loss: 0.5967
Epoch: 006, Loss: 0.6035
Epoch: 007, Loss: 0.6064
Epoch: 008, Loss: 0.6012
Epoch: 009, Loss: 0.5946
Epoch: 010, Loss: 0.5903
Epoch: 011, Loss: 0.5882
Epoch: 012, Loss: 0.5836
Epoch: 013, Loss: 0.5796
Epoch: 014, Loss: 0.5797
Epoch: 015, Loss: 0.5762
Epoch: 016, Loss: 0.5729
Epoch: 017, Loss: 0.5791
Epoch: 018, Loss: 0.5726
Epoch: 019, Loss: 0.5733
Epoch: 020, Loss: 0.5797
Epoch: 021, Loss: 0.5744
Epoch: 022, Loss: 0.5717
Epoch: 023, Loss: 0.5627
Epoch: 024, Loss: 0.5662
Epoch: 025, Loss: 0.5670
Epoch: 026, Loss: 0.5697
Epoch: 027, Loss: 0.5665
Epoch: 028, Loss: 0.5622
Epoch: 029, Loss: 0.5592
Epoch: 030, Loss: 0.5582
Epoch: 031, Loss: 0.5591
Epoch: 032, Loss: 0.5527
Epoch: 033, Loss: 0.5603
Epoch: 034, Loss: 0.5512
Epoch: 035, Loss: 0.5508
Epoch: 036, Loss: 0.5499
Epoch: 037, Loss: 0.5481
Epoch: 038, Loss: 0.5432
Epoch: 039, Loss: 0.5496


In [101]:
gcn_embeddings = model_gcn(data.x, data.edge_index).detach().cpu().numpy()

In [102]:
gcn_embeddings

array([[-1.8652898],
       [-2.0220864],
       [-1.8491397],
       ...,
       [-1.7761192],
       [-1.6381758],
       [-1.8900195]], dtype=float32)

In [103]:
labels = data.y.detach().cpu().numpy()

In [104]:
probs=torch.sigmoid(model_gcn(data.x, data.edge_index))

In [105]:
predictions = (probs > 0.5).long()

In [178]:
from sklearn.cluster import KMeans

In [27]:
from sklearn.cluster import AgglomerativeClustering

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [106]:
accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average=None)
recall = recall_score(labels, predictions, average=None)
f1 = f1_score(labels, predictions, average=None)

In [33]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")

Accuracy: 0.7662650602409639
Precision: [0.7982906 0.590625 ]
Recall: [0.91449086 0.3480663 ]
F1 score: [0.85244904 0.43800695]


In [107]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")

Accuracy: 0.759277108433735
Precision: [0.78816634 0.57671958]
Recall: [0.92167102 0.30110497]
F1 score: [0.84970663 0.39564428]


# USING Class Weighting

In [76]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [77]:
labels = data.y.cpu().numpy() 
classes = np.unique(labels)

In [116]:
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=labels)

In [117]:
weights = torch.tensor(class_weights, dtype=torch.float) 

In [118]:
criterion2 = torch.nn.BCEWithLogitsLoss(pos_weight=weights[1])


In [119]:
def train2(model, dataset, optimizer, criterion, epochs=50):
    if len(dataset.train_mask.shape) > 1:
        train_mask = dataset.train_mask[:, 0]
    else:
        train_mask = dataset.train_mask

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        # out = model(dataset.x, dataset.edge_index)  # Output shape: [N, 2]
        # loss = criterion(out[train_mask], dataset.y[train_mask])  # Shapes: [N_train, 2] and \[N_train]
        out = model(dataset.x, dataset.edge_index, dataset.edge_weight).squeeze()  # Output shape: [num_nodes]
        loss = criterion(out[train_mask], dataset.y[train_mask].float())  
        
        loss.backward()
        optimizer.step()

        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')


In [120]:
train2(model=model_gcn, dataset=data,
      criterion=criterion2, optimizer=optimizer,
      epochs=50)

Epoch: 000, Loss: 0.8822
Epoch: 001, Loss: 0.8827
Epoch: 002, Loss: 0.8828
Epoch: 003, Loss: 0.8829
Epoch: 004, Loss: 0.8820
Epoch: 005, Loss: 0.8824
Epoch: 006, Loss: 0.8816
Epoch: 007, Loss: 0.8828
Epoch: 008, Loss: 0.8837
Epoch: 009, Loss: 0.8825
Epoch: 010, Loss: 0.8836
Epoch: 011, Loss: 0.8822
Epoch: 012, Loss: 0.8818
Epoch: 013, Loss: 0.8818
Epoch: 014, Loss: 0.8812
Epoch: 015, Loss: 0.8818
Epoch: 016, Loss: 0.8825
Epoch: 017, Loss: 0.8819
Epoch: 018, Loss: 0.8820
Epoch: 019, Loss: 0.8823
Epoch: 020, Loss: 0.8815
Epoch: 021, Loss: 0.8821
Epoch: 022, Loss: 0.8827
Epoch: 023, Loss: 0.8826
Epoch: 024, Loss: 0.8832
Epoch: 025, Loss: 0.8824
Epoch: 026, Loss: 0.8824
Epoch: 027, Loss: 0.8814
Epoch: 028, Loss: 0.8822
Epoch: 029, Loss: 0.8827
Epoch: 030, Loss: 0.8816
Epoch: 031, Loss: 0.8819
Epoch: 032, Loss: 0.8832
Epoch: 033, Loss: 0.8826
Epoch: 034, Loss: 0.8822
Epoch: 035, Loss: 0.8815
Epoch: 036, Loss: 0.8821
Epoch: 037, Loss: 0.8822
Epoch: 038, Loss: 0.8816
Epoch: 039, Loss: 0.8823


In [121]:
probabilities = torch.sigmoid(model_gcn(data.x, data.edge_index)).squeeze()
predictions2 = (probabilities > 0.5).long()

In [122]:
predictions2 = predictions2.cpu().numpy()

In [123]:
accuracy = accuracy_score(labels, predictions2)
precision = precision_score(labels, predictions2, average=None)
recall = recall_score(labels, predictions2, average=None)
f1 = f1_score(labels, predictions2, average=None)

In [41]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.7602409638554217
Precision: [0.83686096 0.54216867]
Recall: [0.83877285 0.53867403]
F1 Score: [0.83781581 0.5404157 ]


In [124]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.2612048192771084
Precision: [0.         0.26133076]
Recall: [0.         0.99815838]
F1 Score: [0.         0.41421475]


## FAKE WEIGHTS

In [87]:
weights[1]

tensor(1.9107)

In [88]:
fake_weight=torch.tensor(2.5)

In [89]:
criterion3 = torch.nn.BCEWithLogitsLoss(pos_weight=fake_weight)


In [90]:
train2(model=model_gcn, dataset=data,
      criterion=criterion3, optimizer=optimizer,
      epochs=50)

Epoch: 000, Loss: 0.8760
Epoch: 001, Loss: 0.8475
Epoch: 002, Loss: 0.8552
Epoch: 003, Loss: 0.8509
Epoch: 004, Loss: 0.8515
Epoch: 005, Loss: 0.8597
Epoch: 006, Loss: 0.8483
Epoch: 007, Loss: 0.8585
Epoch: 008, Loss: 0.8648
Epoch: 009, Loss: 0.8415
Epoch: 010, Loss: 0.8422
Epoch: 011, Loss: 0.8548
Epoch: 012, Loss: 0.8448
Epoch: 013, Loss: 0.8604
Epoch: 014, Loss: 0.8491
Epoch: 015, Loss: 0.8452
Epoch: 016, Loss: 0.8487
Epoch: 017, Loss: 0.8586
Epoch: 018, Loss: 0.8624
Epoch: 019, Loss: 0.8445
Epoch: 020, Loss: 0.8388
Epoch: 021, Loss: 0.8555
Epoch: 022, Loss: 0.8540
Epoch: 023, Loss: 0.8532
Epoch: 024, Loss: 0.8429
Epoch: 025, Loss: 0.8584
Epoch: 026, Loss: 0.8514
Epoch: 027, Loss: 0.8592
Epoch: 028, Loss: 0.8484
Epoch: 029, Loss: 0.8436
Epoch: 030, Loss: 0.8495
Epoch: 031, Loss: 0.8462
Epoch: 032, Loss: 0.8428
Epoch: 033, Loss: 0.8470
Epoch: 034, Loss: 0.8442
Epoch: 035, Loss: 0.8506
Epoch: 036, Loss: 0.8413
Epoch: 037, Loss: 0.8414
Epoch: 038, Loss: 0.8412
Epoch: 039, Loss: 0.8415


In [91]:
probabilities = torch.sigmoid(model_gcn(data.x, data.edge_index)).squeeze()
predictions3 = (probabilities > 0.5).long()

In [92]:
predictions3 = predictions3.cpu().numpy()


In [93]:
accuracy = accuracy_score(labels, predictions3)
precision = precision_score(labels, predictions3, average=None)
recall = recall_score(labels, predictions3, average=None)
f1 = f1_score(labels, predictions3, average=None)

In [49]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.739277108433735
Precision: [0.85724585 0.50145349]
Recall: [0.77610966 0.63535912]
F1 Score: [0.81466256 0.5605199 ]


In [94]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.6785542168674699
Precision: [0.87510841 0.43275488]
Recall: [0.65861619 0.73480663]
F1 Score: [0.75158287 0.5447099 ]


In [50]:
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
import networkx as nx

In [53]:
G = to_networkx(data, to_undirected=True)


In [54]:
nx.write_gexf(G,"graph_data_with.gexf")